In [26]:
import pandas as pd
import numpy as np

dataset = pd.read_csv('export/dataset.csv', delimiter=',', on_bad_lines='skip')
print(len(dataset))
dataset = dataset.drop(columns=['title', 'director', 'resume'])
dataset = dataset[dataset['genre'] != np.nan ]
dataset = dataset.dropna(subset=['year'])
dataset = dataset.dropna(subset=['imdbRating'])
dataset = dataset.dropna(subset=['parentalAdvisor'])
dataset = dataset.dropna(subset=['duree'])
dataset = dataset.dropna(subset=['genre'])
dataset = dataset[dataset['themaScore'] != 0 ]
dataset = dataset.reindex()
print(len(dataset))
print(dataset)

<ipython-input-26-00e25d1523fe>:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('export/dataset.csv', delimiter=',', on_bad_lines='skip')


3042340
146925
         year  imdbRating parentalAdvisor  duree      genre  themaScore
1        2001         5.8       Not Rated   95.0      Crime    0.050744
2        2011         5.9       Not Rated   94.0      Crime    0.204428
3        2013         5.7       Not Rated  111.0     Action    0.051400
7        2018         6.7         Unrated  151.0  Biography    0.187660
9        1998         5.5       Not Rated   50.0  Adventure    0.188830
...       ...         ...             ...    ...        ...         ...
3028074  2019         5.8           TV-14   88.0     Comedy    0.282121
3028086  2009         5.2           PG-13   88.0  Biography    0.208951
3028087  2009         5.2           PG-13   88.0     Comedy    0.105011
3028099  2019         8.5       Not Rated   90.0     Action    0.373675
3028105  2017         8.2       Not Rated   92.0      Drama    0.117758

[146925 rows x 6 columns]


In [27]:
target_name = 'imdbRating'

In [28]:
# ReIndex the dataset for put the TARGET on last column. 
dataset = dataset.reindex(columns = [col for col in dataset.columns if col != target_name] + [target_name])

# Suppression des Nan sur la TARGET.
dataset = dataset.dropna(subset=[target_name])

print('Valeurs manquantes en % :')
100*dataset.isnull().sum()/dataset.shape[0]

Valeurs manquantes en % :


year               0.0
parentalAdvisor    0.0
duree              0.0
genre              0.0
themaScore         0.0
imdbRating         0.0
dtype: float64

## Graph de correlation 

In [29]:
import plotly.figure_factory as ff

corr_matrix = round(dataset.corr(), 4)

fig = ff.create_annotated_heatmap(corr_matrix.values,
                                  x = corr_matrix.columns.values.tolist(),
                                  y = corr_matrix.index.values.tolist(),
                                  )


fig.show()

## Separation Varibles / Target

In [31]:
## Separation valeurs explicative et valeur cible
Y = dataset[:][target_name]
X = dataset.drop(columns= [target_name])

## Separation en train/test
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15, random_state=0)
# stratify = Y # option pour même ratio de réponse que sur la table Y.
print(X)
print(Y)

         year parentalAdvisor  duree      genre  themaScore
1        2001       Not Rated   95.0      Crime    0.050744
2        2011       Not Rated   94.0      Crime    0.204428
3        2013       Not Rated  111.0     Action    0.051400
7        2018         Unrated  151.0  Biography    0.187660
9        1998       Not Rated   50.0  Adventure    0.188830
...       ...             ...    ...        ...         ...
3028074  2019           TV-14   88.0     Comedy    0.282121
3028086  2009           PG-13   88.0  Biography    0.208951
3028087  2009           PG-13   88.0     Comedy    0.105011
3028099  2019       Not Rated   90.0     Action    0.373675
3028105  2017       Not Rated   92.0      Drama    0.117758

[146925 rows x 5 columns]
1          5.8
2          5.9
3          5.7
7          6.7
9          5.5
          ... 
3028074    5.8
3028086    5.2
3028087    5.2
3028099    8.5
3028105    8.2
Name: imdbRating, Length: 146925, dtype: float64


## Pipeline Préprocessor 

In [32]:
## Médiane quantitative & plus fréquent qualitatif

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import( OneHotEncoder, StandardScaler, LabelEncoder )
from sklearn.compose import ColumnTransformer

# Create pipeline for numeric features
numeric_features = X.select_dtypes([np.number]).columns # Automatically detect positions of numeric columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), # missing values will be replaced by columns' median
    ('scaler', StandardScaler())
])

# Create pipeline for categorical features
categorical_features = X.select_dtypes("object").columns # Automatically detect positions of categorical columns
categorical_transformer = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # missing values will be replaced by most frequent value
    ('encoder', OneHotEncoder(drop='first')) # first column will be dropped to avoid creating correlations between features
    ])

# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

X_train = preprocessor.fit_transform(X_train) # Preprocessing influenceur
X_test = preprocessor.transform(X_test) # Preprocessing copieur


## Training
### Random Forest

In [33]:
# https://machinelearningmastery.com/multi-core-machine-learning-in-python/
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_jobs=-1, verbose=2) 
# n_jobs=-1 -> Utilisation de tout les cores 
# verbose=2 -> Affiche toutes les info de constructions

## Prediction
### Random Forest model : Grid search

In [42]:
from sklearn.model_selection import GridSearchCV
# Grid of values to be tested
params = {
    'max_depth': [4, 6, 8, 10],
    'min_samples_leaf': [1, 2, 5],
    'min_samples_split': [2, 4, 8],
    'n_estimators': [10, 20, 40, 60, 80, 100]
}
gridsearch = GridSearchCV(model, param_grid = params, cv = 3) # cv : the number of folds to be used for CV
gridsearch.fit(X_train, Y_train)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_regression.py:796: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_regression.py:796: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_regression.py:796: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_regression.py:796: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_regression.py:796: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.

/Library/F

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [4, 6, 8, 10],
                         'min_samples_leaf': [1, 2, 5],
                         'min_samples_split': [2, 4, 8],
                         'n_estimators': [10, 20, 40, 60, 80, 100]})

In [43]:
Y_train_pred = gridsearch.predict(X_train) # Predictions on training set
Y_test_pred = gridsearch.predict(X_test) # Prédictions on test set 

#### Grid search : Random Forest

In [44]:
from sklearn.metrics import ( accuracy_score, f1_score, r2_score )
from sklearn import metrics

print("r2-score on training set : ", r2_score(Y_train, Y_train_pred))
print("r2-score on test set : ", r2_score(Y_test, Y_test_pred))

r2-score on training set :  0.6947368421052642
r2-score on test set :  -13.728888888888916


## Save Model

In [50]:
import joblib
# save
joblib.dump(gridsearch.best_estimator_, "export/random_forest.joblib", compress=3)

['export/random_forest.joblib']

In [ ]:
# load, no need to initialize the loaded_rf
# loaded_rf = joblib.load("./random_forest.joblib")

# loaded_rf.predict(X)